In [39]:
import re
import base64

In [40]:
#converting a string to binary
def str_to_binary(text):
    return ' '.join(format(ord(i), '08b') for i in text)

In [64]:
#converting a binary to string
def binary_to_str(binary_text):
    """Convert binary string to readable text, ensuring correct 8-bit chunks."""
    binary_values = binary_text.split()  # Split into separate binary characters

    # Validate 8-bit chunks
    valid_binary_values = [b for b in binary_values if len(b) == 8 and set(b) <= {"0", "1"}]

    if len(valid_binary_values) != len(binary_values):  # If any chunk is invalid
        return "ERROR: Invalid Binary Format"

    decoded_text = ''.join(chr(int(b, 2)) for b in valid_binary_values)  # Convert to text
    #print(f"Decoded Message: {repr(decoded_text)}")  # Debugging Step
    return decoded_text

In [42]:
#wrapping a string - adds a special unicode character(\ufeff - zero width non breaking space) at the beginning and end of every string
def wrap(text):
    return "\ufeff" + text + "\ufeff"

In [62]:
def unwrap(string):
    """Extract hidden data wrapped inside \ufeff markers."""
    if string.startswith("\ufeff") and "\ufeff" in string[1:]:  # Ensure it has both markers
        extracted = string[1:string.rindex("\ufeff")]  # Extract between markers
        #print(f"Extracted Hidden Data (repr): {repr(extracted)}") 
        return extracted
    print("No hidden message found in unwrap()! Exiting function.")
    return None  # No hidden message found

In [63]:
#convert binary to zero width characters
def binary_to_hidden(text):
    return (text.replace(' ', '\u2060') #word joiner
            .replace('0', '\u200B') #zero width space
            .replace('1', '\u200C')) 


In [65]:
def hidden_to_binary(text):
    binary_text = (text.replace("\u2060", ' ')  # WORD JOINER (U+2060)
                              .replace("\u200B", '0')  # ZERO WIDTH SPACE (U+200B)
                              .replace("\u200C", '1')) # ZERO WIDTH NON-JOINER (U+200C)
    #print(f"Extracted Binary (repr): {repr(binary_text)}")
    
    # Ensure proper spacing between binary chunks
    binary_list = binary_text.split()
    if any(len(b) != 8 for b in binary_list):
        print(f"Warning: Binary chunks not aligned as 8-bits")  # Debugging step
    return ' '.join(binary_list)  # Ensure proper spacing

In [ ]:
def encode_stego_file(input_file, secret_message, output_file):
    with open(input_file, "r", encoding="utf-8") as file:
        cover_text = file.read()
    print(f"Text present in original file: {cover_text}")

    binary_secret = str_to_binary(secret_message)
    hidden_data = binary_to_hidden(binary_secret)
    stego_text = wrap(hidden_data) + cover_text

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(stego_text)
    
    print(f"Stego file saved as {output_file}")

secret_message = "Vadlamudi Jyothsna"
encode_stego_file("stego_file.txt", secret_message, "stego_output.txt")

Text present in cover file: Hello World, this is our SE project called ShroudX, group 17. @Pradeepika, Jyothsna, Manideep, Vivekananda, Sriya. This is a steganography based project. Hope you like it :)
Stego file saved as stego_output.txt


In [70]:
def decode_stego_file(stego_file, output_file):
    #print("Opening stego file..")
    try:
        with open(stego_file, "r", encoding="utf-8") as file:
            stego_text = file.read()
        print(f"Text present in stego file: {stego_text}")
            
        #print("Stego file read successfully!")
        #print(f"Raw Content (repr): {repr(stego_text)}")
    except Exception as e:
        print(f"Error reading stego file: {e}")
        return

    #print("Extracting hidden data...")
    extracted_hidden = unwrap(stego_text)
    if not extracted_hidden:
        #print("No hidden message found! Exiting function.")
        return  # Exit if no hidden message is extracted

    extracted_binary = hidden_to_binary(extracted_hidden)
    decoded_message = binary_to_str(extracted_binary)

    #print("Attempting to save decoded message...")
    try:
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(decoded_message)
        print(f"Decoded message saved as {output_file}")  #success
    except Exception as e:
        print(f"ERROR saving file: {e}")

# Example
decode_stego_file("stego_output.txt", "decoded_text.txt")
with open("decoded_text.txt", "r", encoding="utf-8") as file:
    text = file.read()
print(f"Decoded message: {text}")

Text present in stego file: ﻿​‌​‌​‌‌​⁠​‌‌​​​​‌⁠​‌‌​​‌​​⁠​‌‌​‌‌​​⁠​‌‌​​​​‌⁠​‌‌​‌‌​‌⁠​‌‌‌​‌​‌⁠​‌‌​​‌​​⁠​‌‌​‌​​‌⁠​​‌​​​​​⁠​‌​​‌​‌​⁠​‌‌‌‌​​‌⁠​‌‌​‌‌‌‌⁠​‌‌‌​‌​​⁠​‌‌​‌​​​⁠​‌‌‌​​‌‌⁠​‌‌​‌‌‌​⁠​‌‌​​​​‌﻿Hello World, this is our SE project called ShroudX, group 17. @Pradeepika, Jyothsna, Manideep, Vivekananda, Sriya. This is a steganography based project. Hope you like it :)
Decoded message saved as decoded_text.txt
Decoded message: Vadlamudi Jyothsna
